In [ ]:
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
import pandas as pd

def read_the_file(**kwargs):
    filepath = "/opt/airflow/sales/sales.csv"
    df = pd.read_csv(filepath)
    print(df)
    kwargs['ti'].xcom_push(key='read_the_file', value=df)

def write_csv_file(**kwargs):
    output_file_location = "/opt/airflow/Newfolder/output_file.json"
    print(output_file_location)
    
    df=kwargs['ti'].xcom_pull(task_ids='read_the_file', key='read_the_file')

    if df is not None:
        print('The given link is valid and can be executed')
        df.to_csv(output_file_location, index=True)
    else:
        print('Error in the given file')    

default_args ={
    'owner':'ravi',
    'description':'this dag will read the file',
    'retries':2,
    'retry_delay':timedelta(minutes=2),
}

with DAG(
    dag_id='reading_sales_file_dagg_01',
    default_args=default_args,
    schedule_interval='@once',
    start_date=datetime.now(),
    catchup=False
) as dag:


    read_file = PythonOperator(
        task_id='read_the_file',
        python_callable=read_the_file,
        dag=dag,   
    )

    write_file = PythonOperator(
        task_id='writing_the_file_to_json',
        python_callable=write_csv_file,
        provide_context=True,
        dag=dag,
    )

    read_file >> write_file